# テキストの埋め込みを使った類似性検索デモノートブック
埋め込みモデル：Cohere Embed Multilingual v 3.0(OCI Generative AI Searvice)

In [61]:
import oci
import os
from dotenv import load_dotenv, find_dotenv
import numpy as np
import pandas as pd

In [62]:
_= load_dotenv(find_dotenv())

## OCI認証パラメータの設定

In [63]:
CONFIG_PROFILE = "DEFAULT" # 構成ファイルに合わせて変更してください。
config = oci.config.from_file(file_location='~/.oci/config', profile_name=CONFIG_PROFILE)
config["region"] = "us-chicago-1"

## OCI GenAIクライアントの生成

In [64]:
compartment_id = os.getenv("OCI_COMPARTMENT_ID") 

In [65]:
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))

## 埋め込み処理共通のパラメータの設定

In [66]:
model_id = os.getenv("OCI_GENAI_EMBED_MODEL_ID")
print(f"model_id:{model_id}")

model_id:cohere.embed-multilingual-v3.0


In [67]:
embed_text_detail = oci.generative_ai_inference.models.EmbedTextDetails()
embed_text_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id=model_id)
embed_text_detail.compartment_id = compartment_id

## 検索対象ドキュメントの埋め込み処理

### 検索対象のドキュメントの定義

In [68]:
documents = [
    "COVID-19には多くの症状があります。",
    "COVID-19の症状は悪いです。",
    "COVID-19の症状は良くありません",
    "COVID-19の症状はいろいろです",
    "COVID-19の症状は悪いです。5Gの機能には、より広範なサービスカバレッジ、より多くの利用可能な接続数、および低電力消費が含まれます。",
    "COVID-19はウイルスによって引き起こされる病気です。最も一般的な症状は発熱、悪寒、および喉の痛みですが、他にもさまざまな症状があります。",
    "COVID-19の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。",
    "認知症には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。",
    "COVID-19には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。",
    "COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。",
    "新型コロナ感染症の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。",
    "鳥インフルエンザの症状には、高熱、咳、筋肉の痛み、倦怠感、頭痛、その他の症状が含まれます。",
    "COVID-19は、SARS-CoV-2（新型コロナウイルス）によって引き起こされる感染症です。2019年12月に最初に確認され、その後、世界的なパンデミックとなりました。症状としては、発熱、咳、息切れ、倦怠感、嗅覚や味覚の喪失などがあります。また、感染経路は、主に飛沫感染と接触感染ですが、空気感染の可能性もあります。高齢者や基礎疾患のある方は重症化リスクが高く、死亡率も高いとされています。COVID-19に対しては、ワクチン接種や手洗い、マスク着用、ソーシャルディスタンスなどの予防措置が推奨されています。パンデミックは世界中の医療システムや経済に大きな影響を与えました。",
    "SGDs（持続可能な開発目標）は、2015年に国連加盟国によって採択された「持続可能な開発のための2030アジェンダ」の中核を成す17の国際目標です。",
    "ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。",
    "著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。",
    "ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。",
    "近似最近傍検索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。",
    "Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです"
]

In [69]:
embed_text_detail.inputs = documents

### ドキュメントの埋め込み実行

In [70]:
embed_text_detail.input_type = "SEARCH_DOCUMENT" # 検索対象のドキュメント
doc_embedding = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
doc_embedding_vectors = np.array([embed for embed in doc_embedding.data.embeddings])

In [71]:
print(f"埋め込みの数:{len(doc_embedding.data.embeddings)}")
print(f"埋め込みの次元数):{len(doc_embedding.data.embeddings[0])}")

埋め込みの数:19
埋め込みの次元数):1024


In [72]:
print(f"1件目の埋め込み:{doc_embedding.data.embeddings[0]}")

1件目の埋め込み:[-0.0001835823, -0.0014657974, 0.03866577, -0.0029411316, 0.006828308, 0.05480957, 0.011764526, -0.025314331, -0.016326904, -0.016494751, 0.03765869, -0.0013818741, 0.011016846, -0.0030822754, -0.0009326935, -0.0015869141, 0.02532959, -0.007007599, 0.06304932, 0.031433105, -0.0026741028, 0.00087690353, 0.013885498, -0.0848999, 0.01838684, 0.03265381, 0.041107178, -0.027938843, 0.023147583, -0.017562866, -0.029876709, -0.021102905, 0.026473999, -0.0076179504, -0.015136719, -0.030441284, 0.010025024, -0.0592041, -0.009063721, 0.050628662, 0.026809692, -0.027648926, -0.03527832, 0.038970947, -0.00843811, -0.016983032, 0.04248047, 0.05130005, 0.047973633, 0.03314209, -0.029312134, 0.007850647, 0.016586304, -0.025909424, 0.01675415, -0.011016846, 0.006969452, 0.028335571, 0.021377563, 0.037750244, 0.013076782, -0.016189575, -0.046203613, 0.0029087067, 0.007320404, 0.013519287, -0.003698349, 0.03012085, -0.011772156, 0.014129639, 0.02696228, 0.011497498, 0.047943115, -0.013748169, -

## 類似性検索のデモ（1）

### クエリーの埋め込み処理

#### クエリーの定義

In [73]:
query1 = ["ベクトルデータベースとは？"]

#### クエリー1の埋め込み生成

In [74]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query1
query_embedding1 = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
query_vector1 = np.array(query_embedding1.data.embeddings[0])

### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [75]:
# 類似度スコアの計算
scores = np.dot(query_vector1, np.transpose(doc_embedding_vectors))

#### 結果の表示（上位5件）

In [76]:
# スコアとドキュメントをデータフレームに変換
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.8247,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
1,2,0.7664,著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。
2,3,0.6599,ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。
3,4,0.5685,Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです
4,5,0.4050,COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。


## 類似性検索のデモ（2）

### クエリーの埋め込み処理

#### クエリーの定義

In [77]:
query2 = ["近似最近傍検索とは？"]

#### クエリー2の埋め込み生成

In [78]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query2
query_embedding2 = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
query_vector2 = np.array(query_embedding2.data.embeddings[0])

### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [79]:
# 類似度スコアの計算
scores = np.dot(query_vector2, np.transpose(doc_embedding_vectors))

#### 結果の表示（上位5件）

In [80]:
# スコアとドキュメントをデータフレームに変換
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.8964,近似最近傍検索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。
1,2,0.6872,ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。
2,3,0.4275,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
3,4,0.3810,著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。
4,5,0.3495,Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです


## 類似性検索のデモ（3） - おまけ（クエリーの埋め込みの平均ベクトルで検索してみる）

### クエリーの埋め込み処理

#### クエリーの定義

In [81]:
query1 = ["ベクトルデータベースとは？"]
query2 = ["近似最近傍検索とは？"]

#### クエリー1、2の埋め込み実行

In [82]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query1
query_embedding1 = generative_ai_inference_client.embed_text(embed_text_detail)
embed_text_detail.inputs = query2
query_embedding2 = generative_ai_inference_client.embed_text(embed_text_detail)

#### クエリー1、2の埋め込みの平均を計算

In [83]:
query_vector1 = np.array(query_embedding1.data.embeddings[0])
query_vector2 = np.array(query_embedding2.data.embeddings[0])
avg_query_vector = (query_vector1 + query_vector2) / 2
print(f"埋め込みの平均のノルム（ベクトルの長さ）: {np.linalg.norm(avg_query_vector)}")
query_embedding_vector = avg_query_vector / np.linalg.norm(avg_query_vector)
print(f"正規化した埋め込みのノルムベクトルの長さ）: {np.linalg.norm(query_embedding_vector)}")

埋め込みの平均のノルム（ベクトルの長さ）: 0.8370894665025899
正規化した埋め込みのノルムベクトルの長さ）: 0.9999999999999999


### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [84]:
# レスポンスオブジェクトから埋め込みベクトルを抽出
doc_embedding_vectors = np.array([embed for embed in doc_embedding.data.embeddings])
# 類似度スコアの計算
scores = np.dot(query_embedding_vector, np.transpose(doc_embedding_vectors))

#### 結果の表示（上位5件）

In [85]:
# スコアとドキュメントをデータフレームに変換
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.8044,ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。
1,2,0.7725,近似最近傍検索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。
2,3,0.7482,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
3,4,0.6855,著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。
4,5,0.5486,Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです


## 類似性検索のデモ（4） - クロスリンガル（言語跨りの）検索

### クエリーの埋め込み処理

#### クエリーの定義

In [86]:
query1 = ["What is a vector database?"]
#query1 = ["¿Qué es una base de datos vectorial?"] #スペイン語

#### クエリー1の埋め込み生成

In [87]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query1
query_embedding1 = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
query_vector1 = np.array(query_embedding1.data.embeddings[0])

### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [88]:
# 類似度スコアの計算
scores = np.dot(query_vector1, np.transpose(doc_embedding_vectors))

#### 結果の表示（上位5件）

In [89]:
# スコアとドキュメントをデータフレームに変換
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.5701,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
1,2,0.5260,著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。
2,3,0.4734,Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです
3,4,0.4087,ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。
4,5,0.3015,COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。


## 類似性検索のデモ（5） - ドキュメントの品質の考慮

### クエリーの埋め込み処理

#### クエリーの定義

In [90]:
query1 = ["COVIT-19の症状は？"]

In [91]:
documents

['COVID-19には多くの症状があります。',
 'COVID-19の症状は悪いです。',
 'COVID-19の症状は良くありません',
 'COVID-19の症状はいろいろです',
 'COVID-19の症状は悪いです。5Gの機能には、より広範なサービスカバレッジ、より多くの利用可能な接続数、および低電力消費が含まれます。',
 'COVID-19はウイルスによって引き起こされる病気です。最も一般的な症状は発熱、悪寒、および喉の痛みですが、他にもさまざまな症状があります。',
 'COVID-19の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。',
 '認知症には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。',
 'COVID-19には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。',
 'COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。',
 '新型コロナ感染症の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。',
 '鳥インフルエンザの症状には、高熱、咳、筋肉の痛み、倦怠感、頭痛、その他の症状が含まれます。',
 'COVID-19は、SARS-CoV-2（新型コロナウイルス）によって引き起こされる感染症です。2019年12月に最初に確認され、その後、世界的なパンデミックとなりました。症状としては、発熱、咳、息切れ、倦怠感、嗅覚や味覚の喪失などがあります。また、感染経路は、主に飛沫感染と接触感染ですが、空気感染の可能性もあります。高齢者や基礎疾患のある方は重症化リスクが高く、死亡率も高いとされています。COVID-19に対しては、ワクチン接種や手洗い、マスク着用、ソーシャルディスタンスなどの予防措置が推奨されています。パンデミックは世界中の医療システムや経済に大きな影響を与えました。',
 'SGDs（持続可能な開発目標）は、2015年に国連加盟国によって採択された「持続可能な開発のための2030アジェンダ」の中核を成す17の国際目標です。',
 'ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索

#### クエリー1の埋め込み生成

In [92]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query1
query_embedding1 = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
query_vector1 = np.array(query_embedding1.data.embeddings[0])

### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [93]:
# 類似度スコアの計算
scores = np.dot(query_vector1, np.transpose(doc_embedding_vectors))

#### 結果の表示（上位5件）

In [94]:
# スコアとドキュメントをデータフレームに変換
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.7850,COVID-19の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。
1,2,0.7780,COVID-19には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。
2,3,0.7703,COVID-19はウイルスによって引き起こされる病気です。最も一般的な症状は発熱、悪寒、および喉の痛みですが、他にもさまざまな症状があります。
3,4,0.7299,COVID-19は、SARS-CoV-2（新型コロナウイルス）によって引き起こされる感染症です。2019年12月に最初に確認され、その後、世界的なパンデミックとなりました。症状としては、発熱、咳、息切れ、倦怠感、嗅覚や味覚の喪失などがあります。また、感染経路は、主に飛沫感染と接触感染ですが、空気感染の可能性もあります。高齢者や基礎疾患のある方は重症化リスクが高く、死亡率も高いとされています。COVID-19に対しては、ワクチン接種や手洗い、マスク着用、ソーシャルディスタンスなどの予防措置が推奨されています。パンデミックは世界中の医療システムや経済に大きな影響を与えました。
4,5,0.7249,COVID-19には多くの症状があります。
